In [1]:
import tensorflow as tf
import numpy as np
import datasets
import keras

In [2]:
dataset_train = datasets.load_dataset("Dataset/Train")

Resolving data files:   0%|          | 0/61003 [00:00<?, ?it/s]

In [3]:
dataset_test = datasets.load_dataset("Dataset/Test")

Resolving data files:   0%|          | 0/8990 [00:00<?, ?it/s]

In [4]:
dataset_train = dataset_train['train'].shuffle()
dataset_train = dataset_train.train_test_split(test_size=0.1)
dataset_val = dataset_train['test']
dataset_train = dataset_train['train']

dataset_train = (dataset_train).with_format("tf")
dataset_val = (dataset_val).with_format("tf")

dataset_test = (dataset_test['train']).with_format("tf")

In [5]:
print(dataset_val.shape)
print(dataset_train.shape)
print(dataset_test.shape)

(6101, 2)
(54902, 2)
(8990, 2)


In [6]:
X_val = dataset_val['image']
y_val = dataset_val['label']

X_train = dataset_train['image']
y_train = dataset_train['label']

X_test = dataset_test['image']
y_test = dataset_test['label']

In [7]:
X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.int32).flatten()
X_test = np.asarray(X_test, dtype=np.float32)
y_test = np.asarray(y_test, dtype=np.int32).flatten()
X_val = np.asarray(X_val, dtype=np.float32)
y_val = np.asarray(y_val, dtype=np.int32).flatten()

In [8]:
print(X_test.shape)

(8990, 64, 64, 3)


In [104]:
@keras.saving.register_keras_serializable(package="SleepPoseClassification")
class SleepNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, channel_3, fc1, num_classes):
        keras.Input((64, 64, 3))
        super(SleepNet, self).__init__()

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        
        self.conv11 = tf.keras.layers.Conv2D(channel_1, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv12 = tf.keras.layers.Conv2D(channel_1, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv21 = tf.keras.layers.Conv2D(channel_2, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv22 = tf.keras.layers.Conv2D(channel_2, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv31 = tf.keras.layers.Conv2D(channel_3, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        self.conv32 = tf.keras.layers.Conv2D(channel_3, (3,3), activation='relu', padding = 'same',
                                   kernel_initializer=initializer)
        
        self.maxpool = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        
        self.fc1 = tf.keras.layers.Dense(fc1, kernel_initializer=initializer,activation='relu')
        
        
        self.fc2 = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer)
        
        self.softmax = tf.keras.layers.Softmax()
    
    def call(self, input_tensor, training=False):
        
        x = self.conv11(input_tensor)
        x = self.conv12(x)
        
        x = self.maxpool(x)
        x = self.conv21(x)
        x = self.conv22(x)
        
        x = self.maxpool(x)
        x = self.conv31(x)
        x = self.conv32(x)
        
        x = self.maxpool(x)
        x = self.flatten(x)
        
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.softmax(x)       

        return x
    
    # def build(self, input_shape):
    # 
    #     self.conv11.build((None,64,64,3))
    #     self.conv12.build((None,64,64,32))
    #     self.conv21.build((None,32,32,64))
    #     self.conv22.build((None,32,32,64))
    #     self.conv31.build((None,16,16,128))
    #     self.conv32.build((None,16,16,128))
    #     self.flatten.build((None,8192))
    #     self.fc1.build((None,512))
    #     self.fc2.build((None,3))

In [105]:
# v1
# channel_1 = 16
# channel_2 = 32
# channel_3 = 64
# fc1 = 512
# num_classes = 5

In [106]:
channel_1 = 32
channel_2 = 64
channel_3 = 128
fc1 = 512
num_classes = 3

In [117]:
model = SleepNet(channel_1, channel_2, channel_3, fc1, num_classes)
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.summary()

Model: "sleep_net_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_108 (Conv2D)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_109 (Conv2D)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_110 (Conv2D)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_111 (Conv2D)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_112 (Conv2D)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_113 (Conv2D)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_18 (Softmax)            │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [118]:
acc_test = 0
while (acc_test < 0.95):
    model.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val), batch_size=64)
    _, acc_test = model.evaluate(X_test, y_test)
    print(acc_test)
    
    
model.evaluate(X_test,y_test)
model.summary()

  8/858 ━━━━━━━━━━━━━━━━━━━━ 2:35 183ms/step - loss: 3309.4546 - sparse_categorical_accuracy: 0.3361

KeyboardInterrupt: 

In [55]:
model.evaluate(X_val,y_val)

191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 184.8031 - sparse_categorical_accuracy: 0.3175


[186.32041931152344, 0.3206031918525696]

In [113]:
model.build(input_shape=(None,64,64,3))

C:\Users\dboga\PycharmProjects\sleep quality using computer vision\.venv\Lib\site-packages\keras\src\layers\layer.py:396: UserWarning: `build()` was called on layer 'sleep_net_17', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [114]:
model.load_weights('SleepNetModelv2_95.weights.h5')

In [115]:
model.evaluate(X_test,y_test)

281/281 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 67.9416 - sparse_categorical_accuracy: 0.5409


[96.22164916992188, 0.37430477142333984]

In [120]:

model.summary()

Model: "sleep_net_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_108 (Conv2D)             │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_109 (Conv2D)             │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_110 (Conv2D)             │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_111 (Conv2D)             │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_112 (Conv2D)             │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_113 (Conv2D)             │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 3)              │         1,539 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_18 (Softmax)            │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,966,728 (34.21 MB)

 Trainable params: 4,483,363 (17.10 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,483,365 (17.10 MB)